In [1]:
import numpy as np
import setigen as stg
from blimpy import Waterfall
import matplotlib.pyplot as plt
import random
import os
from astropy import units as u
from tqdm import tqdm
from sklearn.metrics import silhouette_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

os.environ["CUDA_VISIBLE_DEVICES"]="1"


2023-08-24 05:13:17.542574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def painting(data):
    all_data = []
    labels = []
    for c in range(num_classes):
        drift = 2*random.random()*(-1)**random.randint(0,2)
        snr = random.randint(100, 150)
        width = random.randint(20, 50)
        for s in range(num_samples_per_class):
            index = random.randint(0, data.shape[0]-1)
            window = data[index, :,:]
            
            start = random.randint(50, 180)
            
            frame = stg.Frame.from_data(df=2.7939677238464355*u.Hz,
                                        dt=18.253611008*u.s,
                                        fch1=1289*u.MHz,
                                        ascending=True,
                                        data=window)
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            all_data.append(frame.data)
            labels.append(c)
    all_data = np.array(all_data)
    labels = np.vstack(labels)
    return all_data, labels

In [4]:
import cv2
import numpy as np
from tqdm import tqdm
import gc
from AE import AE

In [5]:
latent_dim = 10
time_samples = 16
freq_sample =  256
encoder_inputs = keras.Input(shape=(time_samples, freq_sample, 1))
x = layers.Conv2D(3, 3, activation="relu", strides=1, padding="same")(encoder_inputs)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=1, padding="same")(x)
x = layers.MaxPool2D(pool_size=(1, 2))(x)
x = layers.BatchNormalization()(x)
x_shape = x.shape
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.BatchNormalization()(x)
z = layers.Dense(latent_dim, name="latent", activation="linear")(x)
encoder = keras.Model(encoder_inputs, z, name="encoder")
encoder.summary()

2023-08-24 05:13:19.498343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 05:13:19.991713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13888 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:61:00.0, compute capability: 8.6


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 256, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 16, 256, 3)        30        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 128, 3)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 16, 128, 3)       12        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 128, 64)       1792      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 64, 64)       0   

In [6]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64, activation="relu")(latent_inputs)
x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(x_shape[1]* x_shape[2]* x_shape[3], activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Reshape((x_shape[1], x_shape[2], x_shape[3]))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(3, 3, activation="relu", strides=2, padding="same")(x)
x = layers.MaxPool2D(pool_size=(2, 1))(x)
x = layers.BatchNormalization()(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="linear", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense_2 (Dense)             (None, 64)                704       
                                                                 
 dense_3 (Dense)             (None, 256)               16640     
                                                                 
 batch_normalization_7 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 8192)              2105344   
                                                                 
 batch_normalization_8 (Batc  (None, 8192)             32768     
 hNormalization)                                           

In [7]:
from tqdm import tqdm
import gc
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = 8
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(
            name="kl_loss"
        )
        self.kl_additional = tf.keras.losses.KLDivergence()
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
        ]
    def gaussanity_loss(self, data, base):
        return self.kl_additional(data, base)
    
    def train_step(self, data_in):
        data = data_in
        print(data.shape)
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            
            
            total_loss = reconstruction_loss + self.beta * kl_loss
        
        mse_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction), axis=(1, 2)
                )
            )
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reconstruction_loss_tracker.update_state(mse_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
        }
    def test_step(self, data_in):
        data, _ = data_in
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))

        total_loss = reconstruction_loss + self.beta * kl_loss 
        
        mse_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction), axis=(1, 2)
                )
            )
        self.total_loss_tracker.update_state(total_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reconstruction_loss_tracker.update_state(mse_loss)
        return {
            "test_loss": self.total_loss_tracker.result(),
            "test_kl_loss": self.kl_loss_tracker.result(),
            "test_reconstruction_loss": self.reconstruction_loss_tracker.result()
        }
    def __call__ (self, inputs):
        return self.decoder(self.encoder(inputs)[0])

In [8]:
autoencoder = VAE(encoder, decoder)
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate = 1e-3))
autoencoder.load_weights("../autoencoder/models/full-weights-"+'07-02-2023-11-08-47')

In [9]:
def normalize(data):
    epsilon = 1
    min_val = data.min()
    data = data - min_val + epsilon
    new_data = np.log(data)
    min_val = data.min()
    max_val = data.max()
    final_data = (data - min_val) / (max_val - min_val)
    return final_data
    
def normalize_data(data):
    for i in tqdm(range(data.shape[0])):
        data[i,:,:] = normalize(data[i,:,:])
    return data

In [10]:
def idealized(num=10_000):
    drift = 0
    snr = 20
    width = 10
    start = 256//2
    data = []
    labels = []
    for tag in range(4):
        label_vec = np.zeros(4)
        label_vec[tag] = 1
        for i in range(num):
            if tag == 0:
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
            elif tag == 1:
                snr = 50*random.random() +20
            elif tag == 2:
                width = 50*random.random() +20
            elif tag == 3:
                start = random.randint(50, 180)

            frame = stg.Frame(fchans=256*u.pixel,
                              tchans=16*u.pixel,
                              df=2.7939677238464355*u.Hz,
                              dt=18.253611008*u.s,
                              fch1=6095.214842353016*u.MHz)
            noise = frame.add_noise(x_mean=1, noise_type='chi2')
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            data.append(frame.data)
            labels.append(label_vec)
    data = np.array(data)
    labels = np.vstack(labels)
    return data, labels

In [13]:
import os
from tqdm import tqdm
import keras
total_scores = []
for i in tqdm(range(10)):
    idealized_signals, labels = idealized(num = 10_000)
    input_data = np.expand_dims(normalize_data(idealized_signals), axis = -1)
    print(input_data[0,:,:].max(), input_data[0,:,:].min())
    splits = input_data.shape[0]//1_000
    X = []
    print(input_data.shape)
    for i in range(splits):
        tensor = tf.convert_to_tensor(input_data[i*1_000:(i+1)*1_000, :,:,:], dtype=tf.float32)
        X.append(autoencoder.encoder(tensor))
    X = np.vstack(X)
    print(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

    linear = keras.Sequential(name="my_sequential")
    linear.add(layers.Dense(4, activation="sigmoid", name="layer2"))

    linear.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    linear.fit(X_train, y_train, epochs=150, batch_size=128, validation_split = 0.1, shuffle=True)
    results = linear.evaluate(X_test, y_test)
    print(results)
    total_scores.append(results[-1])

100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:03<00:00, 11665.03it/s]


1.0 0.0
(40000, 16, 256, 1)
(40000, 10)
Epoch 1/150
189/189 [==============================] - 2s 9ms/step - loss: 14.1809 - accuracy: 0.3994 - val_loss: 7.9241 - val_accuracy: 0.5224
Epoch 2/150
189/189 [==============================] - 2s 8ms/step - loss: 4.3001 - accuracy: 0.5888 - val_loss: 3.0050 - val_accuracy: 0.6377
Epoch 3/150
189/189 [==============================] - 2s 10ms/step - loss: 2.5753 - accuracy: 0.6599 - val_loss: 2.2501 - val_accuracy: 0.6664
Epoch 4/150
189/189 [==============================] - 1s 3ms/step - loss: 1.8281 - accuracy: 0.6793 - val_loss: 1.5073 - val_accuracy: 0.6847
Epoch 5/150
189/189 [==============================] - 2s 9ms/step - loss: 1.1514 - accuracy: 0.7039 - val_loss: 0.9116 - val_accuracy: 0.7220
Epoch 6/150
189/189 [==============================] - 2s 11ms/step - loss: 0.6988 - accuracy: 0.7589 - val_loss: 0.5736 - val_accuracy: 0.7944
Epoch 7/150
189/189 [==============================] - 2s 11ms/step - loss: 0.4689 - accuracy: 0.82

 10%|████████▌                                                                            | 1/10 [05:07<46:11, 307.99s/it]

[0.10648290812969208, 0.9571969509124756]



100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:03<00:00, 11682.55it/s]


...


100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:03<00:00, 11713.78it/s]


1.0 0.0
(40000, 16, 256, 1)
(40000, 10)
Epoch 1/150
189/189 [==============================] - 2s 10ms/step - loss: 24.6173 - accuracy: 0.2673 - val_loss: 8.1958 - val_accuracy: 0.3261
Epoch 2/150
189/189 [==============================] - 3s 13ms/step - loss: 3.6889 - accuracy: 0.5055 - val_loss: 1.6376 - val_accuracy: 0.6690
Epoch 3/150
189/189 [==============================] - 1s 7ms/step - loss: 1.0474 - accuracy: 0.7752 - val_loss: 0.6544 - val_accuracy: 0.8716
Epoch 4/150
189/189 [==============================] - 1s 8ms/step - loss: 0.4920 - accuracy: 0.8854 - val_loss: 0.3374 - val_accuracy: 0.9108
Epoch 5/150
189/189 [==============================] - 3s 14ms/step - loss: 0.2963 - accuracy: 0.9220 - val_loss: 0.2310 - val_accuracy: 0.9332
Epoch 6/150
189/189 [==============================] - 1s 8ms/step - loss: 0.2367 - accuracy: 0.9382 - val_loss: 0.2067 - val_accuracy: 0.9377
Epoch 7/150
189/189 [==============================] - 3s 14ms/step - loss: 0.2148 - accuracy: 0.9

 60%|███████████████████████████████████████████████████                                  | 6/10 [35:04<23:23, 350.81s/it]


KeyboardInterrupt: 

tensorflow      WARNING  Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).cluster_loss_tracker.total
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).cluster_loss_tracker.count
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.1
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.2
tensorflow      WARNING  Value in checkpoint could not be found in the restored object: (root).optimizer._variables.3
tensorflow      W

In [12]:
print(np.mean(total_scores))
print(np.std(total_scores))

0.9686893999576569
0.008975483173492603
